<a href="https://colab.research.google.com/github/MayerT1/Pakistan_Research/blob/main/Step_1_DL_Training_Chips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://geemap.org/notebooks/96_image_chips/#create-a-function-for-downloading-image

In [1]:
from google.colab import auth
auth.authenticate_user()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install geemap

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [10]:
!pip install retry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 5.5 MB/s eta 0:00:00


In [11]:
import ee
import geemap
import geemap.chart as chart

In [12]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

In [6]:
!pwd

/content


In [7]:
%cd /content/drive/MyDrive/PhD_Main_Folder/Methods/Image_Chips

/content/drive/MyDrive/PhD_Main_Folder/Methods/Image_Chips


In [14]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='servir-ee')
Map = geemap.Map()

In [ ]:
#ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com")

In [15]:
# image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
#             .filterBounds(region) \
#             .filterDate('2021', '2022') \
#             .select('B8', 'B4', 'B3') \
#             .median() \
#             .visualize(min=0, max=4000) \
#             .clip(region)

image = ee.Image('users/tjm0042/PHD/GEDI_Height/stackedResampled');

In [16]:
# # region = Map.user_roi
# region = ee.Geometry.Polygon(
#     [
#         [
#             [-122.513695, 37.707998],
#             [-122.513695, 37.804359],
#             [-122.371902, 37.804359],
#             [-122.371902, 37.707998],
#             [-122.513695, 37.707998],
#         ]
#     ],
#     None,
#     False,
# )

region = image.geometry();

In [17]:
params = {
    "count": 100,  # How many image chips to export
    "buffer": 127,  # The buffer distance (m) around each point
    "scale": 100,  # The scale to do stratified sampling
    "seed": 1,  # A randomization seed to use for subsampling.
    "dimensions": "256x256",  # The dimension of each image chip
    "format": "GEO_TIFF" ,#"png",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    "prefix": "tile_",  # The filename prefix
    "processes": 25,  # How many processes to used for parallel processing
    "out_dir": ".",  # The output directory. Default to the current working directly
}

In [18]:
Map.addLayer(image, {}, "Image")
Map.addLayer(region, {}, "ROI", False)
Map.centerObject(image, 10)
Map

Map(center=[34.10155814460179, 72.70185391752715], controls=(WidgetControl(options=['position', 'transparent_b…

In [19]:
# def getRequests():
#     img = ee.Image(1).rename("Class").addBands(image)
#     points = img.stratifiedSample(
#         numPoints=params["count"],
#         region=region,
#         scale=params["scale"],
#         seed=params["seed"],
#         geometries=True,
#     )
#     Map.data = points
#     return points.aggregate_array(".geo").getInfo()


def getRequests():
    # img = ee.Image(1).rename("Class").addBands(image)
    points = GEDI_Training = ee.FeatureCollection('users/tjm0042/PHD/GEDI_Height/training_GEDI')

    # points = img.stratifiedSample(
    #     numPoints=params["count"],
    #     region=region,
    #     scale=params["scale"],
    #     seed=params["seed"],
    #     geometries=True,
    # )
    Map.data = points
    return points.aggregate_array(".geo").getInfo()

In [20]:
@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
    point = ee.Geometry.Point(point["coordinates"])
    region = point.buffer(params["buffer"]).bounds()

    if params["format"] in ["png", "jpg"]:
        url = image.getThumbURL(
            {
                "region": region,
                "dimensions": params["dimensions"],
                "format": params["format"],
            }
        )
    else:
        url = image.getDownloadURL(
            {
                "region": region,
                "dimensions": params["dimensions"],
                "format": params["format"],
            }
        )

    if params["format"] == "GEO_TIFF":
        ext = "tif"
    else:
        ext = params["format"]

    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    out_dir = os.path.abspath(params["out_dir"])
    basename = str(index).zfill(len(str(params["count"])))
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, "wb") as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)

In [21]:
%%time
logging.basicConfig()
items = getRequests()

pool = multiprocessing.Pool(params["processes"])
pool.starmap(getResult, enumerate(items))

pool.close()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py", line 1335, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 5, in <module>
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 375, in starmap
    return self._map_async(func, iterable, starmapstar, chunksize).get()
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 768, in get
    self.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 765, in wait
    self._event.wait(timeout)
  File "/usr/lib/python3.10/threading.py", line 607, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib/python3.10/threading.py", line 320, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_trac

TypeError: object of type 'NoneType' has no len()

In [ ]:
Map.addLayer(Map.data, {}, "Sample points")
Map

Map(bottom=104929.0, center=[34.10155814460179, 72.70185391752715], controls=(WidgetControl(options=['position…